In [ ]:
# default_exp predict

In [ ]:
from nbdev.showdoc import *

In [ ]:
# export
from grade_classif.data.utils import load_batches
from grade_classif.models.plmodules import GradesClassifModel
from grade_classif.data.read import get_scan

In [ ]:
# export
def predict_one_scan_one_level(model, fn):
    preds = []
    for x in load_batches(fn, bs=model.bs, device=model.main_device):
        preds.append(model.predict(x).detach().cpu()[:, 1])
    return sum(preds)/len(preds)

In [ ]:
# export
def predict_one_scan(hparams):
    preds = []
    for level, version in zip(hparams.levels, hparams.versions):
        hparams.level = level
        model = GradesClassifModel(hparams)
        model.load(version)
        path = get_scan(hparams.data/f'{hparams.data.name}_{level}', hparams.scan, include=['train', 'valid'])
        preds.append(predict_one_scan_one_level(model, path))
    return sum(preds)/len(preds)

In [ ]:
# export
def predict_all(hparams):
    df = pd.read_csv(hparams.data_csv, header='infer')
    preds = []
    for scan in df['scan']:
        hparams.scan = scan
        preds.append(predict_one_scan(hparams))
    return preds, df

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_train.ipynb.
Converted 02_predict.ipynb.
Converted 10_data_read.ipynb.
Converted 11_data_loaders.ipynb.
Converted 12_data_dataset.ipynb.
Converted 13_data_utils.ipynb.
Converted 14_data_transforms.ipynb.
Converted 20_models_pl_modules.ipynb.
Converted 21_models_modules.ipynb.
Converted 22_models_utils.ipynb.
Converted 23_models_hooks.ipynb.
Converted 80_params_defaults.ipynb.
Converted 81_params_norm.ipynb.
Converted 82_params_classif.ipynb.
Converted 83_params_predict.ipynb.
Converted 99_index.ipynb.
